# QLD Conservation Status and Sensitive Species Lists

This notebook downloads the Qld lists from the [Qld Government Open Data Portal](https://data.qld.gov.au) and formats them in Darwin Core for ingestion into the ALA Lists tool.
It will save original lists to the `source-data/QLD` directory, process the lists and save them to `current-lists`.


## Lists in the ALA Species List tool
* Sensitive list: __[dr493](https://lists.ala.org.au/speciesListItem/list/dr493)__ ([dr18404 in test](https://lists-test.ala.org.au/speciesListItem/list/dr18404))
* Collection: __[dr493 in test](https://collections.ala.org.au/public/show/dr493)__

## Source Data
Queensland Nature Conservation Act 1992

* __[Sensitive](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/qld-confidential-species.csv)__
* __[Species Codes](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv)__

**Metadata Description**

**Sensitive:** A list of confidential species for Queensland or "sensitive species" supplied directly by the Queensland Department of Environment and Science. It is published weekly as the Queensland Confidential Species dataset in the Queensland Government Data portal. The Atlas of Living Australia generalises all latitude and longitude values in occurrence records of sensitive species in Queensland to one decimal place. This rule is applied to all Queensland occurrence records regardless of source.

**Metadata URL**
* Qld Species (Open Data Portal) https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife
* Queensland Confidential Species (Open Data Portal) https://www.data.qld.gov.au/dataset/queensland-confidential-species
* Qld Species codes https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife/resource/6344ea93-cadf-4e0c-9ff4-12dfb18d5f14


# Setup
* Import libraries
* Set Project directory
* Set URLs

In [10]:
import datetime

import pandas as pd
import requests
import io
from ftfy import fix_encoding
import urllib.request, json
import certifi
import ssl
import os
import sys

projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
# projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf

sourceDataDir = projectDir + "source-data/QLD/"
statusDir = projectDir + "source-data/status-codes/"
processedDataDir = projectDir + "current-lists/"
state = 'QLD'
codesfile = statusDir + state + "-codes.csv"
codesurl =  "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv"
sensitivelisturl = "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/qld-confidential-species.csv"

## Download the raw files from data.qld.gov.au
... save locally

In [ ]:
# %%script echo skipping # comment this line to download dataset from API

# Status codes
response = requests.get(codesurl)
rtext = fix_encoding(response.text)
speciescodes = pd.read_csv(io.StringIO(rtext))
speciescodes.to_csv(sourceDataDir + "species-status-codes.csv", index=False)

# Confidential/Sensitive List
response = requests.get(sensitivelisturl)
rtext = fix_encoding(response.text)
sensitivelist = pd.read_csv(io.StringIO(rtext))
sensitivelist.to_csv(sourceDataDir + "qld-confidential-species.csv", index=False)


## Conservation List
* Read in the Conservation list
* Join to the codes to expand the code descriptions.
* Change the field names to `sourceStatus` and `status` as required by the ALA's conservation list processing.
* Remove **Least concern**, **Special least concern** and **no status**
* Expand the endemicity and epbc status codes
* Map sourceStatus to status

## Option to read from local files for subsequent runs

In [ ]:
# sensitivelist = pd.read_csv(sourceDataDir + "qld-confidential-species.csv")

## Sensitive - Qld Confidential list
* Read in the Confidential/Sensitive list
* Expand the nca status, endemicity and epbc status codes
* Rename fields to DwC terms
* Replace kingdom and class values with scientific terms


In [12]:
# nca status
# sensitivelist = pd.merge(sensitivelist,ncastatuscodes,left_on=['NCA status'],right_on=['Code'],how="left")
# sensitivelist.drop(['Code'],axis=1,inplace=True)
# sensitivelist = sensitivelist.rename(columns={'NCA status':'sourceStatus','Code_description':'status'})
# # endemicity
# sensitivelist = pd.merge(sensitivelist,endemicitycodes,left_on=['Endemicity'],right_on=['Code'],how="left")
# sensitivelist.drop(['Code','Endemicity'],axis=1,inplace=True)
# sensitivelist = sensitivelist.rename(columns={'Code_description':'Endemicity'})
# # epbc
# sensitivelist = pd.merge(sensitivelist,epbccodes,left_on=['EPBC status'],right_on=['Code'],how="left")
# sensitivelist.drop(['Code','EPBC status','Unnamed: 0'],axis=1,inplace=True)
# sensitivelist = sensitivelist.rename(columns={'Code_description':'EPBC Status'})
sensitivelist.columns

Index(['Taxon Id', 'Kingdom', 'Class', 'Family', 'Scientific name',
       'Common name', 'Taxon author', 'NCA status', 'EPBC status',
       'Significant', 'Endemicity'],
      dtype='object')

In [14]:
# rename fields
sensitivelist = sensitivelist.rename(columns=
{
    'Scientific name':'scientificName',
    'Common name': 'vernacularName',
    'Family': 'family'
})
sensitivelist = sensitivelist[['scientificName', 'vernacularName', 'family']]
sensitivelist['generalisationn'] = "2 km"
# map sourceStatus to category
# sensitivelist['category'] = sensitivelist['sourceStatus']
# sensitivelist.loc[sensitivelist["category"] == "C"] = "LC"
# sensitivelist.loc[sensitivelist["category"] == "CR"] = "CR"
# sensitivelist.loc[sensitivelist["category"] == "E"] = "EN"
# sensitivelist.loc[sensitivelist["category"] == "NT"] = "NT"
# sensitivelist.loc[sensitivelist["category"] == "PE"] = "EW"
# sensitivelist.loc[sensitivelist["category"] == "SL"] = "SL"
# sensitivelist.loc[sensitivelist["category"] == "V"] = "VU"
#
# #sensitivelist.groupby(["kingdom","class"]).size()
#
# sensitivelist.loc[sensitivelist["kingdom"] == "animals", "kingdom"] = "Animalia"
# sensitivelist.loc[sensitivelist["kingdom"] == "plants", "kingdom"] = "Plantae"
# sensitivelist.loc[sensitivelist["class"] == "land plants", "class"] = "Equisetopsida"
# sensitivelist.loc[sensitivelist["class"] == "amphibians", "class"] = "Amphibia"
# sensitivelist.loc[sensitivelist["class"] == "birds", "class"] = "Aves"
# sensitivelist.loc[sensitivelist["class"] == "cartilaginous fishes", "class"] = "Chondrichthyes"
# sensitivelist.loc[sensitivelist["class"] == "insects", "class"] = "Insecta"
# sensitivelist.loc[sensitivelist["class"] == "malacostracans", "class"] = "Malacostraca"
# sensitivelist.loc[sensitivelist["class"] == "mammals", "class"] = "Mammalia"
# sensitivelist.loc[sensitivelist["class"] == "ray-finned fishes", "class"] = "Actinopterygii"
# sensitivelist.loc[sensitivelist["class"] == "reptiles", "class"] = "Reptilia"
# sensitivelist.loc[sensitivelist["class"] == "snails", "class"] = "Gastropoda"
# sensitivelist.loc[sensitivelist["class"] == "arachnids", "class"] = "Arachnida"
sensitivelist

,scientificName,vernacularName,family,generalisationn
0,Rhinolophus philippinensis,greater large-eared horseshoe bat,Rhinolophidae,2 km
1,Chloebia gouldiae,Gouldian finch,Estrildidae,2 km
2,Erythrura trichroa,blue-faced parrot-finch,Estrildidae,2 km
3,Neochmia phaeton evangelinae,crimson finch (white-bellied subspecies),Estrildidae,2 km
4,Poephila cincta cincta,black-throated finch (white-rumped subspecies),Estrildidae,2 km
...,...,...,...,...
954,Pneumatopteris pennigera,lime fern,Thelypteridaceae,2 km
955,Reholttumia costata,NaN,Thelypteridaceae,2 km
956,Thelypteris confluens,NaN,Thelypteridaceae,2 km
957,Macadamia jansenii,NaN,Proteaceae,2 km


In [10]:
# sensitivelist.groupby(["kingdom","class"]).size().sort_values(ascending=False)

kingdom   class         
Plantae   Equisetopsida     851
Animalia  Reptilia           30
          Aves               24
          Amphibia           22
          Malacostraca       10
          Actinopterygii      7
          Insecta             7
          Arachnida           3
          Mammalia            1
dtype: int64

In [15]:
len(sensitivelist.index)

959

## Write to CSV

In [16]:
sensitivelist.to_csv(processedDataDir + "sensitive-lists/QLD-sensitive-20230601.csv",encoding="UTF-8",index=False)
print("Finished writing QLD Sensitive list")